```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
import os
import sys
import pyopencl
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, LxF, HLL, HLL2, KP07, DataOutput, PlotHelper

In [2]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

Using  Intel(R) HD Graphics Haswell Ultrabook GT2 Mobile


In [3]:
#Centered in time, centered in space

nx = 800
ny = 1000

dx = 50000
dy = 50000

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
H0 = 1000.0
A = 0.0 # A is diffusion coefficient multiplied by depth.

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * H0;
eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
v0 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

# Initialization
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_center1 = dy*(ny-100)/2.0
y_center2 = dy*(ny+100)/2.0

for j in range(-1, ny+1):
    for i in range(-1, nx+1):
        x = dx*i - x_center 
        y = dy*j - y_center 
        y1 = dy*j - y_center1
        y2 = dy*j - y_center2

        #Create bump in center of domain for testing
#        size = 50*min(dx, dy)
#        eta0[j+1, i+1] = 0.2*np.exp(-((x/size)**2+(y1/size)**2)) - \
#            0.3*np.exp(-((x/(2*size))**2+(y2/(2*size))**2))

        # Initial smooth step
        inirad = np.sqrt(x**2 + y**2)
        inirad1 = np.sqrt(x**2 + y1**2)
        inirad2 = np.sqrt(x**2 + y2**2)
        L = 15*dx
        D = 50*dx
        etaamp = 0.2
        eta0[j+1, i+1] = 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
        
#        eta0[j+1, i+1] = 0.5*etaamp*(1.0+np.tanh((-inirad1+D)/L)) - \
#            0.5*etaamp*(1.0+np.tanh((-inirad2+D)/L)) 

    
    
#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

# PlotHelper script is in SWEsimulators folder
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

simrange = 300

def animate(i):
    if (i>0):
        t = sim.step(1000.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / simrange) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(simrange), interval=100)
plt.close(anim._fig)
anim

/usr/lib/python2.7/dist-packages/pyopencl/__init__.py:59: CompilerWarning: From-source build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) HD Graphics Haswell Ultrabook GT2 Mobile' on 'Intel Gen OCL Driver' at 0x7efc694eac20> succeeded, but said:

stringInput.cl:185:21: warning: double precision constant requires cl_khr_fp64, casting to single precision
                                                                                                            
  warn(text, CompilerWarning)
/usr/lib/python2.7/dist-packages/pyopencl/__init__.py:59: CompilerWarning: From-source build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) HD Graphics Haswell Ultrabook GT2 Mobile' on 'Intel Gen OCL Driver' at 0x7efc694eac20> succeeded, but said:

stringInput.cl:189:21: warning: double precision constant requires cl_khr_fp64, casting to single precision
                                                                                   

000 % => t=0.0
000 % => t=0.0
003 % => t=10000.0
006 % => t=20000.0
010 % => t=30000.0
013 % => t=40000.0
016 % => t=50000.0
020 % => t=60000.0
023 % => t=70000.0
026 % => t=80000.0
030 % => t=90000.0
033 % => t=100000.0
036 % => t=110000.0
040 % => t=120000.0
043 % => t=130000.0
046 % => t=140000.0
050 % => t=150000.0
053 % => t=160000.0
056 % => t=170000.0
060 % => t=180000.0
063 % => t=190000.0
066 % => t=200000.0
070 % => t=210000.0
073 % => t=220000.0
076 % => t=230000.0
080 % => t=240000.0
083 % => t=250000.0
086 % => t=260000.0
090 % => t=270000.0
093 % => t=280000.0
096 % => t=290000.0


In [ ]:
eta, hu, hv = sim.download()
plt.plot(eta[ny/2,nx/2-250:nx/2+250],'k-x')
plt.ylim((-0.5,0.5))


In [ ]:
def kleinGordonBalance(eta, H, eta_0, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
    c = np.sqrt(g*H)
    a = c/f
    constant = 1/(a*a)
    #print "a: ", a
    #print "dx, dy:", dx, dy
   
    eta_xx = np.zeros_like(eta)
    eta_yy = np.zeros_like(eta)
    laplace_eta = np.zeros_like(eta)
        
    eta_yy[1:-1, :] = (eta[:-2,:] -2*eta[1:-1,:] + eta[2:, :])/(dy*dy)
    eta_xx[:, 1:-1] = (eta[:, :-2] - 2*eta[:, 1:-1] + eta[:, 2:])/(dx*dx)
    laplace_eta = eta_xx + eta_yy
   
    kg = laplace_eta - constant*(eta - eta_0)
    #geoBalance = (Ay + Bx + Cx )/D - 1
    #geoBalance = (Ay + Bx + Cx - D)/(0.5*(Ay + Bx + Cx + D))
    return kg
   
KG_CTCS = kleinGordonBalance(eta[3:-3, 3:-3], H0, \
                              eta0[3:-3, 3:-3], \
                              nx, ny, \
                              dx, dy, \
                              f, g)
plt.imshow(KG_CTCS)
plt.colorbar()